<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/tutorial-pytorch-org/00_5_Optimization_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline 

# Optimizing the model parameters

- Now that we have a model and data it's time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration (called an *epoch*) the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters (as we saw in the module), and **optimizes** these parameters using gradient descent. For a more detailed walkthrough of this process, checkout this video on [backpropogation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

- We load the code from the previous module on **Datasets & DataLoaders** and **Build Model**

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_datasetloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(), 
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Setting hyperparameters

- Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperpameter values can impact model training and coverage rates ([read more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) about hyperparameter tuning)

- We define the following hyperparameters for training: 
  - **Number of Epochs** - the number times to iterate over dataset
  - **Batch Size** - the number of data samples seen by the model in each epoch 
  - **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behaviour during training. 

In [3]:
learning_rate = 1e-3
batch_size = 64
epoch = 5

## Add an optimization loop

- Once we set our hyperparameters, we can then train and optimize our model with an opotimization loop. Each iteration of the optimization loop is called an **epoch**. 

- Each epoch consists of two main parts: 

  - **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters. 
  - ** The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving. 

- Let's briefly familiarize overselves with some of the concepts used in the training loop. Jump ahead to see the `full-impl-label` of the optimization loop. 

## Add a loss function 

- When presented with some training data, our untrained network is likely to give the correct answer. **Loss Function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value. 

- Common loss functions include [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks, and [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines `nn.LogSoftmax` and `nn.NLLLoss`. 

- We pass our model's output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error. 

In [4]:
# Initialize the loss function 
loss_fn = nn.CrossEntropyLoss()

# Optimization pass 
- Optimization is the process of adjusting model parameters to reduce model error in each training step.  **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the `optimizer` object. Here, we use the SGD optimizer; additionally, there are many [different optimizers]() available in PyTorch such as ADAM and RMSProp, that work better for different kinds of model and data. 

- We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter. 

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

- Inside the training loop, optimization happens in three steps: 

  - call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration. 
  - Back-prapagate the prediction loss with a call to `loss.backwards()`. PyTorch deposits the gradients of the loss w.r.t.  each parameter. 
  - Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass. 

## Full implementation
- We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model's performance against our test data. 

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss 
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  

- We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`. Feel free to increase the number of epochs to track the model's improving performance. 

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------")
  train_loop(train_datasetloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------
loss: 2.294447 [    0/60000]
loss: 2.290375 [ 6400/60000]
loss: 2.274692 [12800/60000]
loss: 2.273125 [19200/60000]
loss: 2.280028 [25600/60000]
loss: 2.278574 [32000/60000]
loss: 2.259083 [38400/60000]
loss: 2.263516 [44800/60000]
loss: 2.238649 [51200/60000]
loss: 2.241249 [57600/60000]
Test Error: 
 Accuracy: 41.1%, Avg loss: 0.035075 

Epoch 2
-------------------------
loss: 2.237605 [    0/60000]
loss: 2.231218 [ 6400/60000]
loss: 2.197969 [12800/60000]
loss: 2.189139 [19200/60000]
loss: 2.214705 [25600/60000]
loss: 2.231251 [32000/60000]
loss: 2.177149 [38400/60000]
loss: 2.196630 [44800/60000]
loss: 2.150629 [51200/60000]
loss: 2.154440 [57600/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 0.033642 

Epoch 3
-------------------------
loss: 2.157704 [    0/60000]
loss: 2.145490 [ 6400/60000]
loss: 2.084302 [12800/60000]
loss: 2.059426 [19200/60000]
loss: 2.127253 [25600/60000]
loss: 2.164787 [32000/60000]
loss: 2.058358 [38400/60000]
loss: 2.104

- You may have noticed that the model is initially not very good (that's OK!). Try running the loop for more `epochs` or adjusting the `learning_rate` to a bigger number. It might also be the case that the model configuration we chose might not be optimal for this kind of problem (it isn't). Later course will delve more into model shapes that work for vision problems. 